In [1]:
import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [2]:
url = "https://beta.adalab.es/resources/apis/christmas.json"

datos = requests.get(url)
datos.status_code
datos.content

datos_pelis = datos.json()

In [9]:
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='',
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


Conexión exitosa


In [4]:
try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS christmas"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Query exitosa


In [ ]:
mycursor.execute("USE christmas")

query = '''CREATE TABLE christmas_movies (
    id INT PRIMARY KEY AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    rating VARCHAR(20),
    runtime INT,
    imdb_rating DECIMAL(3,1),
    meta_score INT,
    genre VARCHAR(255),
    release_year INT,
    description TEXT,
    director VARCHAR(255),
    stars TEXT,
    votes VARCHAR(50),
    gross VARCHAR(50),
    img_src VARCHAR(500),
    type VARCHAR(50)
);'''

mycursor.execute(query)

In [ ]:
# Query de inserción
query_insert = """
INSERT INTO christmas_movies (title, rating, runtime, imdb_rating, meta_score, 
                              genre, release_year, description, director, stars, 
                              votes, gross, img_src, type) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


# Convertir DataFrame a lista de tuplas
#pd.notna(): Verifica si el valor no es NaN/null antes de insertarlo
#iterrows(): Itera sobre las filas del DataFrame

# Reemplazar NaN con None para que se inserten como NULL en la BD
# Reemplazar NaN con None de forma más robusta
df_clean = df_christmas.replace({np.nan: None, 'nan': None, 'NaN': None})
# Convertir DataFrame directamente a lista de tuplas
datos = df_clean.values.tolist()

# Ejecutar inserción
mycursor.executemany(query_insert, datos)
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")

print(f"{mycursor.rowcount} registros insertados")

100 registros insertados


In [6]:
# 1. Contar películas por año
query = "SELECT release_year, COUNT(*) as total FROM christmas_movies GROUP BY release_year ORDER BY release_year"

# 2. Promedio de rating por género
query = "SELECT genre, AVG(imdb_rating) as promedio FROM christmas_movies GROUP BY genre"

# 3. Buscar por director
query = "SELECT * FROM christmas_movies WHERE director LIKE %s"
mycursor.execute(query, ('%Curtis%',))

In [12]:
# Top 10 películas mejor valoradas

mycursor = cnx.cursor()
mycursor.execute("USE christmas")

query = """
SELECT title, imdb_rating, release_year, director
FROM christmas_movies
WHERE imdb_rating IS NOT NULL
ORDER BY imdb_rating DESC
LIMIT 10
"""
df_top = pd.read_sql(query, cnx)
print(df_top)

                                               title  imdb_rating  \
0  It's Always Sunny in Philadelphia: A Very Sunn...          9.2   
1                   ABC Stage 67: A Christmas Memory          9.0   
2                              It's a Wonderful Life          8.6   
3  A Kylie Christmas: Live from the Royal Albert ...          8.6   
4                               Anne of Green Gables          8.5   
5                                      Jingle Vingle          8.5   
6                    The Original Christmas Classics          8.4   
7                     Christmas Eve on Sesame Street          8.4   
8                          A Charlie Brown Christmas          8.3   
9                    How the Grinch Stole Christmas!          8.3   

   release_year           director  
0          2005        Fred Savage  
1          1966        Frank Perry  
2          1946        Frank Capra  
3          2015       Paul Dugdale  
4          1985      Megan Follows  
5          2022  M

/var/folders/cr/f3dqyy756z9ftxxsrmgtgjjh0000gn/T/ipykernel_74870/2460016481.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_top = pd.read_sql(query, cnx)


In [13]:
# 2. Películas por década con rating promedio
query = """
SELECT
    FLOOR(release_year/10)*10 as decada,
    COUNT(*) as total_peliculas,
    ROUND(AVG(imdb_rating), 2) as rating_promedio
FROM christmas_movies
WHERE release_year IS NOT NULL
GROUP BY decada
ORDER BY decada DESC
"""
df_decadas = pd.read_sql(query, cnx)
print(df_decadas)

    decada  total_peliculas  rating_promedio
0     2020              282             5.97
1     2010              374             5.93
2     2000               73             6.14
3     1990               39             6.30
4     1980               18             7.04
5     1970               11             7.04
6     1960               11             7.84
7     1950               13             7.00
8     1940               24             7.00
9     1930               13             7.25
10    1920                3             7.10
11    1890                1             6.40


/var/folders/cr/f3dqyy756z9ftxxsrmgtgjjh0000gn/T/ipykernel_74870/1509744527.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_decadas = pd.read_sql(query, cnx)


In [15]:
# 3. Películas de comedia con duración menor a 2 horas
query = """
SELECT title, runtime, imdb_rating, genre
FROM christmas_movies
WHERE genre LIKE '%Comedy%'
AND runtime < 120
ORDER BY imdb_rating DESC
"""
df_comedias = pd.read_sql(query, cnx)
print(df_comedias)

                                                 title  runtime  imdb_rating  \
0    It's Always Sunny in Philadelphia: A Very Sunn...       43          9.2   
1                     ABC Stage 67: A Christmas Memory       51          9.0   
2                       Christmas Eve on Sesame Street       60          8.4   
3                      How the Grinch Stole Christmas!       26          8.3   
4                            A Charlie Brown Christmas       25          8.3   
..                                                 ...      ...          ...   
464                    Kirk Cameron's Saving Christmas       79          1.3   
465                             A Very Covid Christmas       49          NaN   
466                            A Traditional Christmas        6          NaN   
467                                        Wolf-in-law        3          NaN   
468  Proper Manors: Christmas Special: Christmas Pa...       20          NaN   

                         genre  
0     

/var/folders/cr/f3dqyy756z9ftxxsrmgtgjjh0000gn/T/ipykernel_74870/3321679179.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_comedias = pd.read_sql(query, cnx)


In [14]:
# 4. Directores con más películas navideñas
query = """
SELECT 
    director,
    COUNT(*) as num_peliculas,
    ROUND(AVG(imdb_rating), 2) as rating_promedio
FROM christmas_movies
WHERE director IS NOT NULL
GROUP BY director
HAVING num_peliculas > 1
ORDER BY num_peliculas DESC
LIMIT 10
"""
df_directores = pd.read_sql(query, cnx)
print(df_directores)

         director  num_peliculas  rating_promedio
0   David Winning             13             6.59
1  Peter Sullivan             13             5.98
2  Justin G. Dyck             12             5.34
3   Don McBrearty             11             6.24
4   Fred Olen Ray             11             5.30
5    Jake Helgren             11             5.56
6      Ron Oliver              9             6.06
7    Terry Ingram              9             6.44
8  Marita Grabiak              9             5.73
9   Bradley Walsh              8             6.00


/var/folders/cr/f3dqyy756z9ftxxsrmgtgjjh0000gn/T/ipykernel_74870/3268186820.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_directores = pd.read_sql(query, cnx)


In [16]:
# Búsqueda por actor/actriz
query = """
SELECT title, stars, imdb_rating, release_year
FROM christmas_movies
WHERE stars LIKE %s
ORDER BY imdb_rating DESC
"""
mycursor.execute(query, ('%Hugh Grant%',))
resultados = mycursor.fetchall()
for fila in resultados:
    print(fila)

('Love Actually', 'Hugh Grant, Martine McCutcheon, Liam Neeson, Laura Linney, ', Decimal('7.6'), 2003)
